# Imports

* Importy PyTorch: torch, torch.nn i torch.nn.functional używane do podstawowych operacji tensorowych i modułów sieci neuronowych.
* Import math służy do normalizacji pierwiastka kwadratowego w attention.
* Einops rearrange służy do przekształcania i permutacji tensorów w przyjazny dla czytelnika i wydajny sposób.





In [1]:
import torch
import torchvision
import torch.nn as nn
from einops import rearrange
import torch.nn.functional as F
from collections import OrderedDict
from torchvision.models.detection import MaskRCNN
from torchvision.ops.stochastic_depth import StochasticDepth 
from torchvision.models.detection.anchor_utils import AnchorGenerator
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.ops import FeaturePyramidNetwork
from torch.optim.lr_scheduler import CosineAnnealingLR


# Patch Partition + Linear Embedding

---
„Najpierw dzieli wejściowy obraz RGB na nienakładające się patchs za pomocą modułu dzielenia patch, takiego jak ViT. Każda patch jest traktowana jako „token”, a jej cecha jest ustawiana jako konkatenacja surowych wartości RGB pikseli. W naszej implementacji używamy patch o rozmiarze 4 × 4, a zatem wymiar funkcji każdego patcha wynosi 4 × 4 × 3 = 48. Liniowa warstwa osadzania jest stosowana na tej surowej funkcji, aby rzutować ją na dowolny wymiar (oznaczony jako C)”.

---

Gdzie C jest hyperparametrem, który określa wymiar osadzenia. W naszym przypadku C = 96, dla modelu Swin-Transformer(tiny).




![image](../images/Patch_Partition_Linear_Embedding.png)


Podział patchy w stylu ViT i liniowe embeding można zrealizować za pomocą splotu z rozmiarem jądra, krokiem (stride) równym rozmiarowi patcha oraz wyjściowymi kanałami równymi \(C\). Wynikowy tensor ma wymiary \(H/p * W/p * C\), gdzie każdy „token” odpowiada liniowemu przekształceniu pikseli patcha. Wymiar embedings \(C\) to liczba cech (kanałów), które opisują każdą jednostkę w reprezentacji danych. W naszym przypadku \(C = 96\).

Klasa **SwinEmbedding**, dziedzicząca z **nn.Module**, inicjalizuje:
1. Warstwę splotu \(p * p\) (stride \(p\)), z kanałami wyjściowymi \(C\),
2. **LayerNorm** dla wymiaru embeding \(C\),
3. Funkcję aktywacji ReLU.

W metodzie `forward` wejście jest przepuszczane przez splot, przekształcane i permutowane, łącząc \(H, W\) w \(H * W / p^2\), a wymiar osadzania \(C\) przesuwany na końcową pozycję. Na końcu stosowane są normalizacja i ReLU.

In [2]:
class SwinEmbedding(nn.Module):

  """
  input shape -> (b,c,h,w)
  output shape -> (b, h/4 , w/4, C)

  Where:

  b - batch size
  h - height of the image
  w - width of the image
  C - number of channels

  """

  def __init__(self, patch_size = 4, C = 96):
      super().__init__()
      self.linear_embedding = nn.Conv2d(3,C, kernel_size=patch_size, stride=patch_size)
      self.layer_norm = nn.LayerNorm(C)
      self.relu = nn.ReLU() # activation function (not present in the torchvision model)

  
  def forward(self,x):
    x = self.linear_embedding(x) # image partitioning into patches
    x = rearrange(x, 'b c h w -> b h w c')  # change the shape of the tensor
    x = self.layer_norm(x) 
    x = self.relu(x) # activation function (not present in the torchvision model)

    return x



# Patch Merging Layer

![image](../images/hearachical_system.png)

Aby stworzyć hierarchiczną reprezentację, liczba tokenów jest zmniejszana przez warstwy scalania patchy, gdy sieć staje się głębsza. Pierwsza warstwa scalania patch'y łączy cechy każdej grupy 2 × 2 sąsiednich patch'y i stosuje warstwę liniową na 4C-wymiarowych połączonych cechach. Zmniejsza to liczbę tokenów o wielokrotność 2×2 = 4 (2-krotne zmniejszenie rozdzielczości), a wymiar wyjściowy jest ustawiony na 2C.
Inicjalizujemy warstwę liniową z kanałami wejściowymi 4C do kanałów wyjściowych 2C i inicjalizujemy normę warstwy z wyjściowym rozmiarem osadzania. W naszej funkcji forward używamy einops rearrange do zmiany kształtu naszych tokenów z 2x2xC na 1x1x4C. Kończymy, przepuszczając nasze dane wejściowe przez projekcję liniową i normę warstwy.

![image](../images/Patch_mergering.png)


In [3]:
class PatchMerging(nn.Module):

  """
  Reduces tokens by a factor of 4 (2x2 patches) and doubles embedding dimension.


  input shape -> (b h w c)
  output shape -> (b h/2 w/2 C*2)

  Where:

  b - batch size
  c - number of channels
  h - height of the image
  w - width of the image

  """

  def __init__(self, C) -> None:
     super().__init__()
     self.linear_layer = nn.Linear(C*4, C*2) # Doubles the embedding dimension
     self.layer_norm = nn.LayerNorm(2 * C) # Layer normalization

  def forward(self, x):
    x = rearrange(x, 'b (h ph) (w pw) c -> b h w (ph pw c)', ph=2, pw=2) # Merge patches and double the embedding dimension
    x = self.linear_layer(x) 
    x = self.layer_norm(x)
    return x

# Shifted Window Attention Mechanism


Zaczynamy od zainicjowania naszych parametrów embed_dim, num_heads i window_size oraz zdefiniowania dwóch projekcji liniowych. Pierwsza z nich to nasza projekcja z danych wejściowych do zapytań, kluczy i wartości, którą wykonujemy w jednej równoległej projekcji, więc rozmiar wyjściowy jest ustawiony na 3*C. Druga projekcja to projekcja liniowa zastosowana po obliczeniach uwagi. Projekcja ta służy do komunikacji między połączonymi równoległymi wielogłowicowymi jednostkami uwagi.

Rozpoczynamy naszą funkcję do przodu, uzyskując rozmiar naszej głowy, wysokość i szerokość naszego wejścia, ponieważ potrzebujemy tych parametrów do zmiany układu. Następnie wykonujemy projekcję Q,K,V na naszym wejściu o kształcie ((h*w), c) do ((h*w), 3C). Nasz następny krok składa się z dwóch części, w których zmienimy nasze dane wejściowe ((h*w), C*3) na okna i równoległe głowice uwagi do naszych obliczeń uwagi.

Pózniej rozbijamy naszą macierz na 3 macierze Q,K,V i obliczamy uwagę za pomocą standardowego wzoru uwagi:

Formuła self-attention w mechanizmie transformera wygląda następująco:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V
$$



**Obliczanie Attention Scores**

```python
attention_scores = (Q @ K.transpose(4, 5)) / math.sqrt(h_dim)
```

Dla każdego tokena obliczamy podobieństwo (iloczyn skalarny) między wektorem zapytania  $Q$ a wszystkimi kluczami $K$. Następnie dzielimy przez $(\sqrt{d_k})$, aby zachować stabilność gradientów. Pózniej wyniki zmarnalizowane poprzez  dzielienia na $d_k$, gdzie $d_k$ to wymiar wektorów $Q$ i $K$. Dzielimy przez $(\sqrt{d_k})$, aby zachować stabilność gradientów.


**Softmax i kontekst uwagi**

```python
attention = F.softmax(attention_scores, dim=-1) @ V
```

Obliczamy softmax z $( \text{attention\_scores} )$ w celu uzyskania prawdopodobieństw, które określają „na co” dany token zwraca uwagę. Następnie obliczamy „ważoną sumę” wartości $V$ na podstawie macierzy uwagi. Wynikiem jest nowa reprezentacja każdego tokena, wzbogacona o informacje z innych tokenów w oknie.


Ze względu na sposób, w jaki ukształtowaliśmy nasze macierze, obliczenia uwagi w oknach są wykonywane wydajnie równolegle w oknach i głowicach uwagi. Na koniec przestawiamy tensory z powrotem na ((h*w),C) i zwracamy nasze ostateczne przewidywane dane wejściowe.

![image.png](../images/self-attetention.png)

Później wprowadzamy **Shifted Window Attention Mechanism** w Swin Transformerach umożliwiający wymianę informacji między nieprzecinającymi się okienkami poprzez wprowadzenie przesunięcia ich układu w kolejnych warstwach. Przesunięcie to sprawia, że sąsiednie okienka częściowo na siebie nachodzą, co pozwala na przepływ informacji przez ich granice. Przesunięcie jest realizowane wydajnie za pomocą operacji cyklicznej (np. `torch.roll`), która przemieszcza okienka o połowę ich rozmiaru.

Wyzwanie pojawia się w związku z przesunięciem, ponieważ tokeny z różnych okienek mogą zostać przestrzennie źle dopasowane. Aby temu zapobiec, stosuje się maskowanie uwagi, które blokuje interakcje między tokenami nienależącymi do sąsiednich obszarów obrazu. Maski te są zaprojektowane tak, aby uniemożliwić uwzględnianie informacji między regionami niepołączonymi w oryginalnym układzie.

Ten mechanizm nie tylko umożliwia lokalną uwagę w obrębie okienek, ale także wspiera hierarchiczne uczenie cech poprzez tworzenie połączeń między sąsiadującymi okienkami w kolejnych warstwach.

![image.png](../images/shifted_window_attention_mechanism.webp)

In [4]:
class ShiftedWindowMSA(nn.Module):
    def __init__(self, embed_dim, num_heads, window_size=7, mask=False, attention_dropout=0.0, dropout=0.0):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.window_size = window_size
        self.mask = mask # mask (True/False)
        self.qkv = nn.Linear(embed_dim, 3 * embed_dim)
        self.proj = nn.Linear(embed_dim, embed_dim)
        self.attention_dropout = nn.Dropout(attention_dropout)
        self.proj_dropout = nn.Dropout(dropout)
        self.logit_scale = nn.Parameter(torch.log(10 * torch.ones((num_heads, 1, 1))))

        self.relative_embeddings = RelativeEmbeddings(window_size, num_heads)

    def forward(self, input):
        
        B, H, W, C = input.shape

        # pad feature maps to multiples of window size
        pad_r = (self.window_size - W % self.window_size) % self.window_size
        pad_b = (self.window_size - H % self.window_size) % self.window_size
        x = F.pad(input, (0, 0, 0, pad_r, 0, pad_b))
        _, pad_H, pad_W, _ = x.shape
       
        # Cyclic shift
        if self.mask:
            x = torch.roll(x, (-self.window_size//2, -self.window_size//2), dims=(1,2))

        # Partition windows
        num_windows = (pad_H //self.window_size) * (pad_W // self.window_size)
        x = rearrange(
                    x, 
                    'b (h w_h) (w w_w) c -> (b h w) (w_h w_w) c', 
                    w_h=self.window_size, w_w=self.window_size
                )

        # QKV computation
        qkv = F.linear(x, self.qkv.weight)
        qkv = qkv.reshape(x.size(0), x.size(1), 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Calculate attention 
        q = q * (C // self.num_heads) ** -0.5
        attn = q.matmul(k.transpose(-2, -1))

        # Add relative position bias
        relative_position_bias = self.relative_embeddings()
        attn = attn + relative_position_bias 
       
        if self.mask:
            # Create attention mask
            attn_mask = torch.zeros((pad_H, pad_W), device=x.device)
            
            # Generate coordinates for the mask
            for i in range(0, pad_H, self.window_size):
                for j in range(0, pad_W, self.window_size):
                    attn_mask[i:i + self.window_size, j:j + self.window_size] += 1
            
            # Create mask for each window
            attn_mask = rearrange(
                attn_mask, 
                '(h winh) (w winw) -> (h w) (winh winw)', 
                winh=self.window_size, 
                winw=self.window_size
            )

            # Create mask for each window
            attn_mask = attn_mask.unsqueeze(1) - attn_mask.unsqueeze(2)  # Shape: (num_windows, window_size^2, window_size^2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float('-inf')).masked_fill(attn_mask == 0, 0.0)

            # Add a dimension for num_heads
            attn_mask = attn_mask.unsqueeze(1)  # Shape: (num_windows, 1, window_size^2, window_size^2)

            # Broadcast over batch and num_heads
            attn = attn.view(-1, num_windows, self.num_heads, x.size(1), x.size(1))
            attn = attn + attn_mask.unsqueeze(0)  # Broadcasting over batch and num_heads
            attn = attn.view(-1, self.num_heads, x.size(1), x.size(1))


        attn = attn.softmax(dim=-1)
        attn = self.attention_dropout(attn)

        # Attention output
        x = (attn @ v).transpose(1, 2).reshape(B, -1, C) 
        x = self.proj(x)
        x = self.proj_dropout(x)

        # Reverse cyclic shift
        x = rearrange(
            x, 
            'b (h ws1 w ws2) c -> b (h ws1) (w ws2) c', 
            ws1=self.window_size, 
            ws2=self.window_size,
            h = pad_H // self.window_size,
            w = pad_W // self.window_size
        )
        if self.mask: 
            x = torch.roll(x, (self.window_size//2, self.window_size//2), (1,2))

        #unpad features
        x = x[:, :H, :W, :].contiguous()
        return x

# Relative Position Embeddings

**Relative Position Embeddings**  wprowadzają dodatkową macierz biasu do mechanizmu uwagi własnej, aby uwzględnić relacje przestrzenne między tokenami. W obliczeniach uwagi macierz biasu pozycyjnego $B \in \mathbb{R}^{M^2 \times M^2}$ jest dodawana do wyników podobieństwa, co pozwala modelowi lepiej rozumieć strukturę przestrzenną tokenów w obrębie okienka.

Aby zmniejszyć złożoność, pełna macierz $B$ jest generowana z mniejszej macierzy parametrów $\hat{B} \in \mathbb{R}^{(2M-1) \times (2M-1)}$, gdzie $M$ to rozmiar okienka. Wartości w $B$ są wypełniane na podstawie względnych pozycji tokenów w zakresie $[-M+1, M-1]$ wzdłuż każdej osi. Później macierz $B$ jest dzielona na 4 podmacierze, które są dodawane do wyników podobieństwa w celu uwzględnienia relacji przestrzennych w pionie, poziomie i obu kierunkach przekątnych. Modyfikacja tej formuły wygląda następująco:

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^\top}{\sqrt{d_k}}\right)V + B
$$



W odróżnieniu od osadzeń sinusoidalnych, te osadzenia pozycyjne są uczone podczas treningu, co daje większą elastyczność. Dzięki dodaniu tych osadzeń bezpośrednio do wyników iloczynu zapytań i kluczy, model efektywnie uwzględnia informacje o relacjach przestrzennych, zachowując jednocześnie zgodność wymiarów w obliczeniach uwagi. To rozwiązanie umożliwia lepsze odwzorowanie relacji przestrzennych przy użyciu zwartej reprezentacji parametrów.

![image.png](../images/relative_position_embeddings.png)



In [5]:
class RelativeEmbeddings(nn.Module):
    def __init__(self, window_size, num_heads):
        super().__init__()
        self.window_size = window_size  # Size of the window (e.g., 8x8 or 16x16)
        self.num_heads = num_heads  # Number of attention heads

        # Initialize relative coordinates and relative position index
        self.define_relative_position_bias_table()
        self.define_relative_position_index()

    def define_relative_position_bias_table(self):
        # define a parameter table of relative position bias
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros((2 * self.window_size - 1) * (2 * self.window_size - 1), self.num_heads)
        )  # 2*Wh-1 * 2*Ww-1, nH
        nn.init.trunc_normal_(self.relative_position_bias_table, std=0.02)


    def define_relative_position_index(self):
        """
        This method defines the relative position index for each pixel pair in the window.
        It calculates the differences in positions and generates a unique index for each relative position.
        """
        # Generate coordinates for the height and width of the window
        coords_h = torch.arange(self.window_size)
        coords_w = torch.arange(self.window_size)

        # Create a meshgrid for all the coordinates
        coords = torch.stack(torch.meshgrid(coords_h, coords_w, indexing="ij"))

        # Flatten the coordinates into a 2D array
        coords_flatten = torch.flatten(coords, 1)

        # Calculate the relative position by subtracting each pair of coordinates
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()

        # Shift the coordinates to ensure positive indices
        relative_coords[:, :, 0] += self.window_size - 1
        relative_coords[:, :, 1] += self.window_size - 1

        # Scale the coordinates to a larger range (for uniqueness)
        relative_coords[:, :, 0] *= 2 * self.window_size - 1

        # Sum the two coordinate differences to get a unique index
        relative_position_index = relative_coords.sum(-1).flatten()

        # Register the relative position index as a buffer to be used during training
        self.register_buffer("relative_position_index", relative_position_index)

    def forward(self):
      
      
        # Use the relative position index and the relative coordinates table to compute the bias
        relative_position_bias = F.embedding(
            self.relative_position_index,  # Look up bias values from the relative position index
            self.relative_position_bias_table,  # Use the pre-defined relative position bias table
        )

        # Reshape the bias values to match the shape of the attention logits (window_size * window_size, window_size * window_size, num_heads)
        relative_position_bias = relative_position_bias.view(
            self.window_size * self.window_size, self.window_size * self.window_size, self.num_heads
        )

        # Permute the bias to match the attention mechanism (num_heads, window_size * window_size, window_size * window_size)
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous().unsqueeze(0)

        return relative_position_bias

# Transformer Encoder Block

**Transformer Encoder Block** w Swin Transformer jest zgodny z typową architekturą bloku transformera, z tą różnicą, że wykorzystuje mechanizm uwagi w przesuniętych oknach oraz aktywację GELU w wielowarstwowej perceptronie (MLP). Każdy blok kodera składa się z dwóch głównych etapów: obliczania uwagi oraz przekształceń nieliniowych w MLP.

W pierwszym etapie dane wejściowe są normalizowane i przekazywane do mechanizmu uwagi z przesuniętymi oknami (Shifted Window Attention). Mechanizm ten umożliwia komunikację między sąsiednimi oknami, a wynik uwagi jest dodawany jako rezidual do oryginalnych danych.

Następnie dane przechodzą przez drugi etap, który obejmuje normalizację, warstwę MLP rozszerzającą wymiar przestrzeni osadzania czterokrotnie, zastosowanie aktywacji GELU oraz powrót do pierwotnego wymiaru. Wynik jest również sumowany z danymi z poprzedniego etapu, co umożliwia lepsze propagowanie informacji w sieci.

Dodatkowo wprowadzono klasę **AlternatingEncoderBlock**, która grupuje bloki kodera w pary. Pierwszy blok w parze działa na standardowych oknach, a drugi na przesuniętych oknach, co pozwala na efektywne uchwycenie relacji między tokenami w różnych lokalizacjach.

![image.png](../images/transformer_encoder_block.webp)

In [6]:
class SwinEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, window_size, mask, sd_prob=0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(embed_dim)
        self.stochastic_depth = StochasticDepth(sd_prob, "row") # Stochastic Depth with 0.1 probability of dropping out a row for tiny version of Swin Transformer

        self.WMSA = ShiftedWindowMSA(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=mask)
        self.MLP = nn.Sequential(
            nn.Linear(embed_dim, embed_dim*4),
            nn.GELU(),
            nn.Dropout(p=0.1), # Default dropout probability is 0.0 in the torchvision implementation
            nn.Linear(embed_dim*4, embed_dim)
        )

        # Initialization of weights and biases (bias) in linear layers
        for m in self.MLP:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight) # Xavier initialization for weights, which prevents the disappearance or explosion of gradients during training.
                if m.bias is not None:
                    nn.init.normal_(m.bias, std=1e-6) # Set a small offset, to have a small impact in the initial stages of training.

    def forward(self, x):

        # Attention path with pre-normalization
        res1 = x # Save input for the skip connection
        x = self.stochastic_depth(self.WMSA(self.layer_norm(x))) # Attention block with LayerNorm and Stochastic Depth(more efficient than Dropout for training transformers)
        x = res1 + x # Residual connection

        # MLP path with pre-normalization
        res2 = x  # Save intermediate result for skip connection
        x = self.stochastic_depth(self.MLP(self.layer_norm(x))) # MLP block with LayerNorm and Dropout
        x = res2 + x  # Residual connection

        return x

class AlternatingEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, sd_prob, window_size=7):
        super().__init__()
        self.WSA = SwinEncoderBlock(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=False, sd_prob=sd_prob[0])
        self.SWSA = SwinEncoderBlock(embed_dim=embed_dim, num_heads=num_heads, window_size=window_size, mask=True, sd_prob=sd_prob[1])
    
    def forward(self, x):
        return self.SWSA(self.WSA(x))

# Final Swin-Transformer Class

Mając już zaimplementowane wszystkie komponenty Swin-Transformera, możemy stworzyć jego finalną klasę. Struktura modelu opiera się na oryginalnym artykule, uwzględniając odpowiednie bloki kodera, wymiary osadzeń oraz liczbę głów uwagi.

Model zaczyna się od warstwy osadzania (*Embedding Layer*), która przekształca obraz wejściowy w odpowiednią reprezentację. Następnie przechodzi przez cztery etapy obliczeniowe:  
1. **Etap 1**: Alternating Encoder Block z 96 wymiarami osadzania i 3 głowami uwagi.  
2. **Etap 2**: Alternating Encoder Block z 192 wymiarami osadzania i 6 głowami uwagi.  
3. **Etap 3**: Trzy następujące po sobie Alternating Encoder Blocks z 384 wymiarami osadzania i 12 głowami uwagi.  
4. **Etap 4**: Alternating Encoder Block z 768 wymiarami osadzania i 24 głowami uwagi.  

Każdy etap zawiera proces *Patch Merging*, który zmniejsza rozdzielczość przestrzenną danych i zwiększa liczbę wymiarów kanałów. Finalnie, dane wyjściowe mają wymiary `(1, 49, 768)`, gdzie 1 to wymiar partii, 49 to spłaszczona przestrzeń 7x7, a 768 to liczba kanałów reprezentująca wymiar osadzania.

Testując model z obrazem wejściowym o wymiarach `(1, 3, 224, 224)`, możemy potwierdzić, że implementacja działa zgodnie z oczekiwaniami i generuje poprawne dane wyjściowe. Dzięki temu w pełni zaimplementowaliśmy Swin-Transformer w PyTorch od podstaw!

![image.png](../images/all_stages_swin.png)

In [7]:
class SwinTransformer(nn.Module):
    def __init__(self, depth=[2, 2, 6, 2], embed_dim=96, stochastic_depth_prob=0.2, window_size= 7):
        super().__init__()
        self.Embedding = SwinEmbedding()  # Embedding layer

        # Calculate total number of blocks
        total_stage_blocks = sum(depth)
        stage_block_id = 0

        self.stages = nn.ModuleList()

        in_channels = embed_dim
        for i_stage, num_blocks in enumerate(depth):
            temp_sd_prob = []
            for _ in range(num_blocks):
                # Calculate probability for the current layer
                sd_prob = stochastic_depth_prob * float(stage_block_id) / (total_stage_blocks - 1)
                temp_sd_prob.append(sd_prob)
                stage_block_id += 1

            #Add alternating encoder blocks recording to the depth list divided by 2, because each block has 2 sub-blocks
            sd_prob = [temp_sd_prob[i:i+2] for i in range(0, len(temp_sd_prob), 2)]
            for _ in range(int(num_blocks / 2)):
                num_heads = in_channels // 32
                #print(f"AlternatingEncoderBlock({in_channels}, {num_heads}, {sd_prob[0]})") # Debug
                self.stages.append(
                    AlternatingEncoderBlock(in_channels, num_heads, sd_prob[0], window_size=window_size)
                )
                sd_prob.pop(0)

            # Add patch merging layer if this is not the last stage
            if i_stage < len(depth) - 1:
                self.stages.append(PatchMerging(in_channels))
                #print(f"PatchMerging({in_channels})") # Debug
                in_channels *= 2

    def forward(self, x):
        x = self.Embedding(x)
        for stage in self.stages:
            x = stage(x)

        return x


In [8]:
class SwinTransformerMultiStage(nn.Module):
    """
    Subclass (or replacement) of your SwinTransformer that returns
    4 feature maps from each stage: C2, C3, C4, C5.
    """
    def __init__(self, base_swin):
        super().__init__()
        # Copy over the embedding
        self.Embedding = base_swin.Embedding
        # Copy over the entire 'stages' ModuleList
        self.stages = base_swin.stages
        # You already know embed_dim=96 for tiny model, but not strictly needed here

    def forward(self, x):
        # 1) Patch embedding
        x = self.Embedding(x)  # (B, 56*56, 96)

        # -- Stage 1
        x = self.stages[0](x)
        c2 = x
        x = self.stages[1](x)

        # -- Stage 2
        x = self.stages[2](x)
        c3 = x
        x = self.stages[3](x)

        # -- Stage 3
        x = self.stages[4](x)
        x = self.stages[5](x)
        x = self.stages[6](x)
        c4 = x
        x = self.stages[7](x)

        # -- Stage 4
        x = self.stages[8](x)
        c5 = x
        
    
        # Return all 4 feature maps, C2, C3, C4, C5 convert to (B, C, H, W)
        stage_dict = {
            "c2": rearrange(c2, 'B h w c -> B c h w'),
            "c3": rearrange(c3, 'B h w c -> B c h w'),
            "c4": rearrange(c4, 'B h w c -> B c h w'),
            "c5": rearrange(c5, 'B h w c -> B c h w'),
        }
      

        return stage_dict

In [9]:
class SwinFPNBackbone(nn.Module):
    """
    1) Runs the Swin stages -> returns c2..c5
    2) Feeds them into a standard FeaturePyramidNetwork -> returns multi-scale feature maps
    3) That final dict is what Mask R-CNN expects
    """
    def __init__(self, swin_multistage: nn.Module):
        super().__init__()
        self.swin = swin_multistage
        # Suppose we output 256 channels from FPN
        self.fpn = FeaturePyramidNetwork(
            in_channels_list=[96, 192, 384, 768],  # channels in c2..c5
            out_channels=256,
            # extra_blocks=LastLevelMaxPool()  # optional
        )
        self.out_channels = 256  # FPN’s output channels per scale

    def forward(self, x):
        # x is (B,3,H,W)
        # 1) Get raw stage features
        features = self.swin(x)  # e.g. {"c2":(B,96,56,56), "c3":(B,192,28,28), "c4":(B,384,14,14), "c5":(B,768,7,7)}

        # 2) Rename them to match FPN’s expected keys: "0", "1", "2", "3" or something
        #    or you can pass them in as a dict with the same keys but then set in_channels_list accordingly
        fpn_input = {
        "0": features["c2"],
        "1": features["c3"],
        "2": features["c4"],
        "3": features["c5"],
         }

        # 3) Run FPN
        #    This returns a dict of feature maps at different scales (e.g. "res2", "res3", "res4", "res5")
        #    each will have shape (B, 256, H_out, W_out)
        out = self.fpn(fpn_input)
        return out


In [10]:
def build_swin_maskrcnn(num_classes=2, weights_path=None, device="cpu"):
    base_swin = SwinTransformer(depth=[2, 2, 6, 2], embed_dim=96, window_size=7) # Tiny Swin Transformer

    # Load weights if available
    if weights_path is not None:
        print(f"Download from {weights_path}")
        weights = torch.load(weights_path, map_location=device)
        if "model" in weights:  
            weights = weights["model"]
        try:
            base_swin.load_state_dict(weights, strict=False)
            print("Weights loaded successfully")
        except RuntimeError as e:
            print(f"Error loading weights: {e}")

    # Convert it to multi-stage
    multi_stage_swin = SwinTransformerMultiStage(base_swin)
    # Wrap in FPN
    backbone = SwinFPNBackbone(multi_stage_swin)

    # For multi-scale anchors
    anchor_generator = anchor_generator = AnchorGenerator(
    sizes=((32,), (64,), (128,), (256,)),  # 4 "levels"
    aspect_ratios=((0.5, 1.0, 2.0),)*4     # or explicitly write 4 tuples
)


    transform = GeneralizedRCNNTransform(
        min_size=(480, 800), # Multi-scale training: short side 480 to 800
        max_size=1333, # Long side no more than 1333
        image_mean=[0.485, 0.456, 0.406], # Average values updated (ImageNet)
        image_std=[0.229, 0.224, 0.225], # Updated std (ImageNet)
    )

    model = MaskRCNN(
        backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator,
        box_detections_per_img=100,
        image_mean=None,
        image_std=None,
        transform=transform
    )
    # Force it in case older torchvision
    model.transform = transform
    return model

In [11]:
def test():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    weights_path="../swin_tiny_patch4_window7_224.pth"
    model = build_swin_maskrcnn(num_classes=2, weights_path=weights_path, device=device).to(device)
    x = [torch.randn(3, 224, 224, device=device)]
    targets = [{
        "boxes": torch.tensor([[50,50,150,150]], dtype=torch.float32, device=device),
        "labels": torch.tensor([1], device=device),
        "masks": torch.randint(0,2,(1,224,224), device=device, dtype=torch.uint8),
    }]

    model.train()
    losses = model(x, targets)  # forward pass -> dict of losses
    print(losses)  # e.g. { 'loss_classifier':..., 'loss_box_reg':..., ... }

    model.eval()
    with torch.no_grad():
        preds = model(x)  # inference
        print(preds)

if __name__ == "__main__":
    test()

Download from ../swin_tiny_patch4_window7_224.pth
Weights loaded successfully


/tmp/ipykernel_1673203/4294303689.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path, map_location=device)


{'loss_classifier': tensor(0.7760, device='cuda:0', grad_fn=<NllLossBackward0>), 'loss_box_reg': tensor(0.0009, device='cuda:0', grad_fn=<DivBackward0>), 'loss_mask': tensor(9.7223, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_objectness': tensor(0.7176, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), 'loss_rpn_box_reg': tensor(0.0021, device='cuda:0', grad_fn=<DivBackward0>)}
[{'boxes': tensor([[1.5288e+02, 0.0000e+00, 1.5890e+02, 1.2772e+01],
        [1.6698e+02, 5.0085e+00, 1.7270e+02, 2.1216e+01],
        [1.0826e+02, 1.1247e+02, 1.3720e+02, 1.6678e+02],
        [1.0029e+02, 0.0000e+00, 1.0655e+02, 1.1833e+01],
        [1.9340e+02, 1.2333e+02, 2.0226e+02, 1.3272e+02],
        [2.0151e+02, 1.2461e+02, 2.0700e+02, 1.4267e+02],
        [1.4796e+02, 2.6627e+00, 1.5345e+02, 2.0199e+01],
        [6.9436e+01, 1.9463e+02, 8.2234e+01, 2.2373e+02],
        [1.6510e+02, 3.8595e+00, 1.7089e+02, 2.2540e+01],
        [6.1498e+01, 1.572

In [12]:
# code for training the model
import torch
import torch.nn as nn
from lightning import LightningModule

# <-- Import or define your build_swin_maskrcnn function here
# from your_swin_file import build_swin_maskrcnn


class SwinMaskRCNNModule(LightningModule):
    def __init__(self, num_classes=2, lr=1e-4, weights_path=None, device="cpu"):
        """
        num_classes: # of classes (including background). 
                     If you have 1 actual class, use num_classes=2 
                     (class + background).
        """
        super().__init__()
        self.save_hyperparameters()
        self.model = build_swin_maskrcnn(num_classes=num_classes, weights_path=weights_path, device=device)
        self.lr = lr

    def training_step(self, batch, batch_idx):
        """
        Lightning’s hook for a single training batch.

        batch = (images, targets)
            - images: list of Tensors [C,H,W]
            - targets: list of dicts { 'boxes', 'labels', 'masks', etc. }
        """
        images, targets = batch
        # Forward pass in Mask R-CNN returns a dict of losses in training mode
        loss_dict = self.model(images, targets)
        total_loss = sum(loss for loss in loss_dict.values())

        # Log the total loss
        self.log("train_loss", total_loss, prog_bar=True)
        return total_loss

    # def validation_step(self, batch, CosineAnnealingLRbatch_idx):
    #     """
    #     For validation, Mask R-CNN still returns losses if we pass targets.
    #     """
    #     images, targets = batch
    #     loss_dict = self.model(images, targets)
    #     total_loss = sum(loss for loss in loss_dict.values())

    #     self.log("val_loss", total_loss, prog_bar=True)
    #     return total_loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch

        # Force the model into training mode so it *will* return losses
        self.model.train()
        with torch.no_grad():
            loss_dict = self.model(images, targets)

        total_loss = sum(loss for loss in loss_dict.values())
        self.log("val_loss", total_loss, prog_bar=True)

        # Optionally switch back
        self.model.eval()

        return total_loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.lr)
        #scheduler = CosineAnnealingLR(optimizer, T_max=36, eta_min=self.lr * 0.1)

        return optimizer #, scheduler

In [13]:
import os
import numpy as np
from torch.utils.data import Dataset
from torchvision import transforms
import torchvision.transforms.v2 as v2
from PIL import Image
from pycocotools.coco import COCO


class CocoDetectionDataset(Dataset):
    """
    COCO-format dataset returning (image, target) pairs for Mask R-CNN.
    Each target is a dict:
        {
            "boxes": FloatTensor (N,4),
            "labels": Int64Tensor (N,),
            "masks": UInt8Tensor (N,H,W),
            "image_id": IntTensor (1,)
        }
    """

    def __init__(
        self,
        image_dir,         # e.g. BASE_DIR / "dataset/coco10/train2017_subset/images"
        # e.g. BASE_DIR / "dataset/coco10/train2017_subset/coco10_train_annotations.json"
        ann_file,
        transforms=None,
        single_class=False  # Set True to ignore COCO category_id's and treat as one class
    ):
        super().__init__()
        self.image_dir = str(image_dir)
        self.coco = COCO(str(ann_file))
        self.ids = list(sorted(self.coco.imgs.keys()))
        self.single_class = single_class

        # Example basic transform pipeline
        # (You can do advanced augmentations with v2.* or Albumentations)
        self._transforms = transforms

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)
        anns = self.coco.loadAnns(ann_ids)

        # Load image
        img_info = self.coco.imgs[img_id]
        path = img_info['file_name']
        img_path = os.path.join(self.image_dir, path)
        img = Image.open(img_path).convert("RGB")

        # Build up lists of bounding boxes, masks, labels
        boxes = []
        labels = []
        masks = []

        for ann in anns:
            # Convert [x, y, w, h] to [x_min, y_min, x_max, y_max]
            x, y, w, h = ann['bbox']
            x2 = x + w
            y2 = y + h
            boxes.append([x, y, x2, y2])

            # Single-class or actual category_id
            if self.single_class:
                labels.append(1)
            else:
                labels.append(ann["category_id"])

            # Build per-object binary mask
            m = self.coco.annToMask(ann)  # shape: (H, W)
            masks.append(m)

        # If no annotations, create dummy
        if len(boxes) == 0:
            boxes = np.zeros((0, 4), dtype=np.float32)
            labels = np.zeros((0,), dtype=np.int64)
            masks = np.zeros((0, img.height, img.width), dtype=np.uint8)
        else:
            boxes = np.array(boxes, dtype=np.float32)
            labels = np.array(labels, dtype=np.int64)
            masks = np.stack(masks, axis=0).astype(np.uint8)

        target = {}
        target["boxes"] = torch.as_tensor(boxes, dtype=torch.float32)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64)
        target["masks"] = torch.as_tensor(masks, dtype=torch.uint8)
        target["image_id"] = torch.tensor([img_id], dtype=torch.int64)

        if self._transforms:
            # apply the transforms
            img, target = self._transforms(img, target)

        return img, target

In [14]:
def get_train_transforms():
    return v2.Compose([
        v2.ToImage(),
        v2.RandomResizedCrop(size=(224, 224), antialias=True),
        v2.RandomHorizontalFlip(p=0.5),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]),
    ])


def get_val_transforms():
    return v2.Compose([
        v2.ToImage(),
        v2.Resize(size=(224, 224), antialias=True),
        v2.ToDtype(torch.float32, scale=True),
        v2.Normalize(mean=[0.485, 0.456, 0.406],
                     std=[0.229, 0.224, 0.225]),
    ])

In [15]:
from lightning import seed_everything
import os
from pathlib import Path
import torch
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger

# Re-use your BASE_DIR logic if you like
os.chdir("..")
BASE_DIR = Path(os.getcwd()).resolve()
print(BASE_DIR)
# 1) Create the train/val dataset
train_image_dir = BASE_DIR / "dataset/coco10/train2017_subset/images"
train_ann_file = BASE_DIR / \
    "dataset/coco10/train2017_subset/coco10_train_annotations.json"

val_image_dir = BASE_DIR / "dataset/coco10/val2017_subset/images"
val_ann_file = BASE_DIR / "dataset/coco10/val2017_subset/coco10_val_annotations.json"

weight_path = BASE_DIR / "swin_tiny_patch4_window7_224.pth"

train_dataset = CocoDetectionDataset(
    image_dir=train_image_dir,
    ann_file=train_ann_file,
    transforms=get_train_transforms(),
    single_class=False   # or True if you want a single-class approach
)

val_dataset = CocoDetectionDataset(
    image_dir=val_image_dir,
    ann_file=val_ann_file,
    transforms=get_val_transforms(),
    single_class=False
)

# 2) DataLoaders
# Mask R-CNN expects a list of images & targets, so we need a custom collate:


def collate_fn(batch):
    return tuple(zip(*batch))


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=4,  # pick what fits your GPU
    shuffle=True,
    collate_fn=collate_fn
)

val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    collate_fn=collate_fn
)

# 3) Instantiate our LightningModule (Swin + MaskRCNN)
seed_everything(42)

model = SwinMaskRCNNModule(num_classes=26, lr=1e-4, weights_path=weight_path, device="cuda")

# 4) MLFlow Logger
mlf_logger = MLFlowLogger(
    experiment_name="swin_maskrcnn_experiment",
    tracking_uri="http://localhost:5000",
    log_model=True
)

# 5) (Optional) Callbacks
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath="checkpoints/",
    filename="swinmaskrcnn-{epoch:02d}-{val_loss:.4f}"
)

early_stop_callback = EarlyStopping(
    monitor="val_loss",
    patience=5,
    mode="min"
)

# 6) Trainer
trainer = pl.Trainer(
    max_epochs=10,
    accelerator="gpu" if torch.cuda.is_available() else "cpu",
    logger=mlf_logger,
    callbacks=[checkpoint_callback, early_stop_callback]
)

# 7) Fit
trainer.fit(
    model=model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

/home/wladyka/Swin-Transformer
loading annotations into memory...


Seed set to 42


Done (t=0.93s)
creating index...
index created!
loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
Download from /home/wladyka/Swin-Transformer/swin_tiny_patch4_window7_224.pth
Weights loaded successfully


/tmp/ipykernel_1673203/4294303689.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  weights = torch.load(weights_path, map_location=device)
GPU available: True (cuda), use

MlflowException: API request to http://localhost:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=swin_maskrcnn_experiment (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f88267223c0>: Failed to establish a new connection: [Errno 111] Connection refused'))